In [1]:
import numpy as np
import os
import tensorflow as tf
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Dropout
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
import h5py

#Reads hdf5 files
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

#multiplies 2 matricies
def mat_mul(A, B):
    return tf.matmul(A, B)

#adds noice to some random points
def jitter_point_cloud(batch_data, sigma=0.01, clip=0.05):
    B, N, C = batch_data.shape
    print(B , N , C)
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(B, N, C), -1 * clip, clip)
    jittered_data += batch_data
    return jittered_data


Using TensorFlow backend.


In [2]:
# number of points in each sample
num_points = 2048

# number of categories
k = 7 # 0 big parallelepipeds, 1 small parallelepipeds , 2 small spheres, 3 big spheres, 4 cubes, 5 cylinders, 6 pyramids

# define optimizer
adam = optimizers.Adam(lr=0.005, decay=0.7)

# Pointnet Architecture

# Input Transformation net
input_points = Input(shape=(num_points, 3))
x = Convolution1D(64, 1, activation='relu',input_shape=(num_points, 3))(input_points)
x = BatchNormalization()(x)
x = Convolution1D(128, 1, activation='relu')(x)
x = BatchNormalization()(x)
x = Convolution1D(1024, 1, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=num_points)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
input_T = Reshape((3, 3))(x)

# Forward net
g = Lambda(mat_mul, arguments={'B': input_T})(input_points)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
g = BatchNormalization()(g)

# Feature Transform net
f = Convolution1D(64, 1, activation='relu')(g)
f = BatchNormalization()(f)
f = Convolution1D(128, 1, activation='relu')(f)
f = BatchNormalization()(f)
f = Convolution1D(1024, 1, activation='relu')(f)
f = BatchNormalization()(f)
f = MaxPooling1D(pool_size=num_points)(f)
f = Dense(512, activation='relu')(f)
f = BatchNormalization()(f)
f = Dense(256, activation='relu')(f)
f = BatchNormalization()(f)
f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
feature_T = Reshape((64, 64))(f)

# Forward net
g = Lambda(mat_mul, arguments={'B': feature_T})(g)
g = Convolution1D(64, 1, activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(128, 1, activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(1024, 1, activation='relu')(g)
g = BatchNormalization()(g)

# Global_feature
global_feature = MaxPooling1D(pool_size=num_points)(g)

# Point_net_classification
c = Dense(512, activation='relu')(global_feature)
c = BatchNormalization()(c)
c = Dropout(rate=0.7)(c)
c = Dense(256, activation='relu')(c)
c = BatchNormalization()(c)
c = Dropout(rate=0.7)(c)
c = Dense(k, activation='softmax')(c)
prediction = Flatten()(c)

model = Model(inputs=input_points, outputs=prediction)

# Compile classification model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

W0920 15:31:31.604472 140417225975552 deprecation_wrapper.py:119] From /home/ramozz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0920 15:31:31.608540 140417225975552 deprecation_wrapper.py:119] From /home/ramozz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0920 15:31:31.623465 140417225975552 deprecation_wrapper.py:119] From /home/ramozz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0920 15:31:31.980831 140417225975552 deprecation_wrapper.py:119] From /home/ramozz/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placehold

In [4]:
#Load pre-trained weights
from keras.models import load_model

model.load_weights('classification_weights 195.h5')
print("Weights loaded !")

Weights loaded !


In [ ]:
#reads te point cloud to be classified
from pyntcloud import PyntCloud
import numpy as np

test_cloud = []
cls = 0; #point cloud to be analysed

#reads point cloud file to np.array list
def ImportTarget():
    file = "cloud.pcd"
    data = PyntCloud.from_file(file)
    cloud = data.xyz
    test_cloud.append(cloud)
    
#sends predict result
def kerasPredict(order):
    target = np.array(test_cloud[order])
    target = target.reshape(-1, num_points, 3)
    pred = model.predict(target)
    pred = np.squeeze(pred)
    pred = pred.tolist()
    return pred

#selects the three highest values of the predict classes
def maxValues(): 
    maxvalue= 0
    max1 = 0
    maximus = np.empty([3]) # 3 probabilities
    indices = np.empty([2]) # two most important labels
    totaly = 0
    totalc = True
    i= 0
    indice = 0

    for pos in range (0,3):
        for value in result: 
            if maxvalue < value and pos ==0 :
                maxvalue= value
                max1 = value
                indice = i 
            elif maxvalue < value and pos ==1 and value != max1 :
                maxvalue= value
                max2 = value 
                indice= i
            elif totalc:
                totaly = totaly + value 
            i += 1

        totalc = False
        if(pos == 0):   
            maximus[pos] = maxvalue
            indices[pos] = indice
        if(pos == 1):   
            maximus[pos] = maxvalue
            indices[pos] = indice
        if(pos == 2):
            maximus[pos] = totaly
        i = 0
        maxvalue = 0
        indice = 0

    maximus = np.around(maximus, decimals= 2)

    #passing to string only the 3 highest positions
    # 0 parallelepipeds, 1 spheres, 2 cubes, 3 cylinders, 4 pyramids
    total1 = ' '.join(str(maximus[x]) for x in range(0,3)) 
    total2 = ' '.join(str(indices[y]) for y in range(0,2))
    total = total1 + " " +  total2
    return total


Socket successfully created
socket binded to 12348
socket is listening
Got connection from ('127.0.0.1', 40326)
Analyse this pcd
Got connection from ('127.0.0.1', 40328)
Analyse this pcd
Got connection from ('127.0.0.1', 40332)
Analyse this pcd
Got connection from ('127.0.0.1', 40334)
Analyse this pcd
Got connection from ('127.0.0.1', 40336)
Analyse this pcd
Got connection from ('127.0.0.1', 40338)
Analyse this pcd
Got connection from ('127.0.0.1', 40340)
Analyse this pcd
Got connection from ('127.0.0.1', 40342)
Analyse this pcd
Got connection from ('127.0.0.1', 40346)
Analyse this pcd
Got connection from ('127.0.0.1', 40350)
Analyse this pcd


In [ ]:
#TCP python server

# Socket library 
import socket                
  
# Socket object 
s = socket.socket()          
print ("Socket successfully created")
  
# Reserve a port on your computer 
port = 12348              
  
# Binding to the port 
s.bind(('', port))         
print ("socket binded to %s" %(port) )
  
# Socket into listening mode 
s.listen(5)      
print ("socket is listening" )           
  
# True loop until we interrupt it or an error occurs 
while True: 
    # Stays until a connection is established   
    connection, addr = s.accept()  
    print ('Got connection from', addr )
    
    data = connection.recv(16)
    my_decoded_str = data.decode()
    #type(my_decoded_str) # ensure it is string representation
    
    #if some messsage is received
    if data:
        ImportTarget()
        result = kerasPredict(cls)
        cls += 1  #goes for the next pcd
        total = maxValues()
        print(my_decoded_str)
        my_str = total
        my_str_as_bytes = str.encode(my_str)
        type(my_str_as_bytes) # ensure it is byte representation
        data = 0;
        # send a thank you message to the client.  
        connection.send(my_str_as_bytes)

# Close the connection with the client 
connection.close() 